In [45]:
import pdfplumber
import pytesseract
import cv2
from PIL import Image
import numpy as np

## Paths

In [46]:
from pathlib import Path

image_path = Path("../testfiles")

#image_file = image_path / "Tom_Thumb_test_receipt.jpg"

## Helper Functions

In [47]:
def is_scanned_pdf(pdf_path):
    """ Does not

    :param pdf_path: _description_
    :type pdf_path: _type_
    :return: _description_
    :rtype: _type_
    """
    with pdfplumber.open(pdf_path) as pdf:
        first_page = pdf.pages[0]
        if first_page.extract_text() is None:
            return True
        return False
    
def extract_text_from_pdf(pdf_path):
    """ Imported from original code
    
    Not Used yet

    :param pdf_path: _description_
    :type pdf_path: _type_
    :return: _description_
    :rtype: _type_
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            image = page.to_image()
            page_text = pytesseract.image_to_string(image.original)
            text += page_text
    return text

def preprocess_image(image_path):
    """Preprocess the image for better OCR results.

    :param image_path: _description_
    :type image_path: _type_
    :return: _description_
    :rtype: _type_
    """
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #gray.show()
    ##_, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    # Deskewing can be added here with more complex algorithms
    
    # Dilation makes text bolder
    kernel = np.ones((2, 2), np.uint8)
    dilated = cv2.dilate(gray, kernel, iterations=1)    
    return gray

# Load the image and process 

Preprocess and load into a *large* text area

1. Create the list of test files
2. Loop through the list
3. Load the images and preprocess them
4. Create the sentences list
5. Seach the sentances

In [48]:
test_files = [
    #"Market_Street_test_receipt.jpg",
    #"Tom_Thumb_test_receipt.jpg",
    #"Trek_Allen_test Receipt.jpg",
    #"Walmart_test_receipt.png",
    "HEB_test_receipt.jpg",
    "Target_test_receipt.jpg"
]

In [54]:
import re

for file in test_files:
    img = preprocess_image(image_path / file)
    #cv2.imshow("Processed Image", img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #continue
    
    # Perform OCR using pytesseract
    text = pytesseract.image_to_string(img)

    # Print the size of the extracted text buffer
    print(f"Loaded {len(text)} characters from {file}")

    sentences = re.split(r'\n+', text.strip())
    print(sentences)
    print(f"found {len(sentences)} sentences in {file}")
    date = extract_dates(sentences)
    if date:
        print(f"Found date(s) in {file}: {date}")
    amount = extract_amounts(sentences)
    if amount:
        print(f"Found amount(s) in {file}: {amount}")
    print(f"======\n")    

Loaded 934 characters from HEB_test_receipt.jpg
['| ILLY DARK ROAST WHL BEAN F 13.64', '2 NAVITAS O MACA POWDER oF 13,39 Q', '3 BULK BEETS', '1.40 Lbs @ 1/ 1,68 FW 2.35', '4 JUMBO YELLOW ONION', '0,83 Lbs @ 1/1412 FH 0,98', '5 BULK SWEET POTATOES', '0.77 lbs @ 1/ 1.24 FH 0.95', '6 ALIVE ONCE DAILY ULTRA 15.28 @', '7 BV PITTED MEDJOOL DATES FW 5,98', '8 AUSTINUTS RAW TREE MIX F 15,26', '9 MIXED HABANERO PEPPERS FW 3.46', '10 HUMAN N SUPERBEET HEART C 39.98', '11 ORGANIC BANANAS', '2.47 los @ 1/  O./4 FH 1.83', '_keeeeeepperes® Total Sales 113.09.', 'MASTERCARD', 'FEREEKEKERER GIS |', 'Contactless Chip USD$ 113,09', 'Appr No : 659522 Ref No : 671592', 'Mode: Issuer Verified By Pin', 'MASTERCD', 'AID : A0000000041010', 'TVR + 0000008001', 'TAD + 0114A14003020000000700000000000000FF', 'TSI ARC + 00', 'Returns can be done at any checkstand', 'HEB Food-Drugs 796', '575 E Exchange Pkwy, Allen, TX 75002', 'Phone: (469) 656-9401', 'Pharmacy (214) 509-9043', 'Store Hours: 6 A.M, to 11 P.M', 'You

In [ ]:
for s in sentences:
    print(f"Sentence: {s} | Length: {len(s)}")

## Analyze the extracted text

1. Look for a date and time

### Libary of functions to extract data

One function per data item

In [55]:
import datetime
def extract_dates(sentences):
    """Extract dates from a list of sentences.

    Attempts to find and parse dates in various formats.

    :param sentences: Sentences from the reciept
    :type sentences: List[str]
    :return: date
    :rtype: datetime.date
    """
    for s in sentences:
        if matches := re.findall(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4} \d{2}:\d{2}:\d{2}', s):
            print("1")
            format_string = '%m/%d/%y %H:%M:%S'
            break
        elif matches := re.findall(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4} \d{1,2}:\d{2} [AP]M?', s):
            print("2")
            format_string = '%m/%d/%Y %I:%M %p'
            break
        elif matches := re.findall(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4} \d{1}:\d{2}[AP]', s):
            # Very specific format 
            print("3")
            format_string = '%m/%d/%y %I:%M %p'
            break
        elif matches := re.findall(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4} \d{1,2}:\d{2}', s):
            print("4")
            format_string = '%m/%d/%y %H:%M'
            break
        else:
            continue
        
    if matches:
        print(f"Found date(s) in sentence: {s} | Matches: {matches}")
        try:
            date = [datetime.datetime.strptime(date, format_string) for date in matches]
            return date
        except ValueError as e:
            print(f"Error parsing dates: {e}")
            return None
        
    else:
        print(f"no dates found")
        return None

In [44]:
def extract_amounts(sentences):
    """ Extract the amount of the voucher from the list of sentences.

    Attempts to find and parse amounts in various formats.

    :param sentences: Sentences from the receipt
    :type sentences: List[str]
    :return: amount
    :rtype: float   
    """
    for s in sentences:
        if matches := re.findall(r'\b(Total|TOTAL|Total:|TOTAL:|Balance|BALANCE)\b.*?(\d+(?:,\d{3})*(?:\.\d{2})?)', s):
            print("1")
            break
        #elif matches := re.findall(r'\d+(?:,\d{3})*(?:\.\d{2})? USD', s):
        #    print("2")
        #    break
        #elif matches := re.findall(r'\d+(?:,\d{3})*(?:\.\d{2})?', s):
        #    print("3")
        #    break
        else:
            continue

    if matches:
        print(f"Found amount(s) in sentence: {s} | Matches: {matches}")
        try:
            voucher_amt = float(matches[0][1].replace('$', '').replace(',', ''))
            #print(voucher_amt)
            return voucher_amt
        except ValueError as e:
            print(f"Error parsing amounts: {e}")
            return None

    else:
        print(f"no amounts found")
        return None

In [18]:
import re

s = 'Thursday'
matches = re.findall(r'\b(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)\b', s)
print(matches)

['Thursday']
